In [1]:
import interfaces as inter
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
import numpy as np
nltk.download('wordnet')
nltk.download('stopwords')
from os import listdir
import pandas as pd

[nltk_data] Downloading package wordnet to /home/nikita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nikita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
file = 'corpus.txt'
is_wiki = True
if is_wiki:
    words_GT = np.load('result/words_WIKI_GT.txt',allow_pickle=True)
    counts_GT = np.load('result/counts_WIKI_GT.txt',allow_pickle=True)
else:
    words_GT = np.load('result/words_GT.txt',allow_pickle=True)
    counts_GT = np.load('result/counts_GT.txt',allow_pickle=True)

times = []
ns = []
n_uses = []
n_keeps = []
n_final_tops = []
ws = []
cs = []
heat = []
score = []

In [7]:
#t = %timeit -o !mpiexec —allow-run-as-root -n 4 python main.py
#will not work now, functions should be implemented first!
for n_use in [2000,3000,4000,6000,8000,10000]:
    for n_keep in [n_use]:
        for n_final_top in [1000]:
            for n in [4]:
                inter.file_to_n_files(file,n)
                ns.append(n)
                n_uses.append(n_use)
                n_keeps.append(n_keep)
                n_final_tops.append(n_final_top)
                f = open('main.py','w')
                f.write('''import interfaces as inter
from mpi4py import MPI
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

N_USE = n_use
N_KEEP = n_keep
N_FINAL_TOP = n_final_top
FILENAME = 'corpus'

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
filename = FILENAME + '_0' + str(rank) + '.txt'

inter.pipeline(filename, N_USE, N_KEEP, rank, comm,N_FINAL_TOP,result_suff='_1')'''.replace('n_use',str(n_use)).replace('n_keep',str(n_keep)).replace('n_final_top',str(n_final_top)))
                f.close()
                if n == 1:
                    t = %timeit -o !mpiexec -n 1 python main.py
                elif n == 2:
                    t = %timeit -o !mpiexec -n 2 python main.py
                elif n == 3:
                    t = %timeit -o !mpiexec -n 3 python main.py
                elif n == 4:
                    t = %timeit -o !mpiexec -n 4 python main.py
                times.append(t.average)
                words = np.load('result/words_1.txt',allow_pickle=True)
                counts = np.load('result/counts_1.txt',allow_pickle=True)
                ws.append(words)
                cs.append(counts)
                h,s = inter.accuracy(words_GT, counts_GT, words, counts)
                heat.append(h)
                score.append(s)
                result = pd.DataFrame({'cores':ns,
              'time':times,
              'n_use':n_uses,
              'n_keep':n_keeps,
              'n_final_top':n_final_tops,'words':ws,'counts':cs,'heat rate':heat,'score':score})
                display(result)

31.5 s ± 1.95 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


,cores,time,n_use,n_keep,n_final_top,words,counts,heat rate,score
0,3,36.621663,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.821494,0.932762
1,3,45.874564,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.895735,0.971726
2,3,52.085524,4000,4000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.949318,0.987154
3,3,56.643347,6000,6000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.986097,0.996006
4,3,67.565361,8000,8000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.992032,0.997737
5,3,76.931090,10000,10000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.998002,0.999415
6,2,49.996770,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.792115,0.928155
7,1,79.453068,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.780944,0.936372
8,2,58.160973,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.877934,0.965114
9,1,97.058281,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.904762,0.975089


44.9 s ± 7.11 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


,cores,time,n_use,n_keep,n_final_top,words,counts,heat rate,score
0,3,36.621663,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.821494,0.932762
1,3,45.874564,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.895735,0.971726
2,3,52.085524,4000,4000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.949318,0.987154
3,3,56.643347,6000,6000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.986097,0.996006
4,3,67.565361,8000,8000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.992032,0.997737
5,3,76.931090,10000,10000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.998002,0.999415
6,2,49.996770,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.792115,0.928155
7,1,79.453068,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.780944,0.936372
8,2,58.160973,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.877934,0.965114
9,1,97.058281,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.904762,0.975089


48 s ± 5.35 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


...

,cores,time,n_use,n_keep,n_final_top,words,counts,heat rate,score
0,3,36.621663,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.821494,0.932762
1,3,45.874564,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.895735,0.971726
2,3,52.085524,4000,4000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.949318,0.987154
3,3,56.643347,6000,6000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.986097,0.996006
4,3,67.565361,8000,8000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.992032,0.997737
5,3,76.931090,10000,10000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.998002,0.999415
6,2,49.996770,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.792115,0.928155
7,1,79.453068,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.780944,0.936372
8,2,58.160973,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.877934,0.965114
9,1,97.058281,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.904762,0.975089


53.8 s ± 927 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,cores,time,n_use,n_keep,n_final_top,words,counts,heat rate,score
0,3,36.621663,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.821494,0.932762
1,3,45.874564,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.895735,0.971726
2,3,52.085524,4000,4000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.949318,0.987154
3,3,56.643347,6000,6000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.986097,0.996006
4,3,67.565361,8000,8000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.992032,0.997737
5,3,76.931090,10000,10000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.998002,0.999415
6,2,49.996770,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.792115,0.928155
7,1,79.453068,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.780944,0.936372
8,2,58.160973,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.877934,0.965114
9,1,97.058281,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.904762,0.975089


1min 4s ± 700 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,cores,time,n_use,n_keep,n_final_top,words,counts,heat rate,score
0,3,36.621663,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.821494,0.932762
1,3,45.874564,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.895735,0.971726
2,3,52.085524,4000,4000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.949318,0.987154
3,3,56.643347,6000,6000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.986097,0.996006
4,3,67.565361,8000,8000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.992032,0.997737
5,3,76.931090,10000,10000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.998002,0.999415
6,2,49.996770,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.792115,0.928155
7,1,79.453068,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.780944,0.936372
8,2,58.160973,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.877934,0.965114
9,1,97.058281,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.904762,0.975089


In [9]:
result = pd.DataFrame({'cores':ns,
              'time':times,
              'n_use':n_uses,
              'n_keep':n_keeps,
              'n_final_top':n_final_tops,'words':ws,'counts':cs,'heat rate':heat,'score':score})

In [10]:
result.sort_values('time').head(30)

,cores,time,n_use,n_keep,n_final_top,words,counts,heat rate,score
18,4,31.515223,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.853568,0.940820
0,3,36.621663,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.821494,0.932762
21,4,44.051241,6000,6000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.982161,0.995674
19,4,44.906104,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.912046,0.975130
1,3,45.874564,3000,3000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.895735,0.971726
20,4,48.015434,4000,4000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.956947,0.988938
6,2,49.996770,2000,2000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.792115,0.928155
2,3,52.085524,4000,4000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.949318,0.987154
22,4,53.777900,8000,8000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18486, 18167, 18021, 15908, 15173, 144...",0.996008,0.999015
3,3,56.643347,6000,6000,1000,"[people, also, use, make, one, river, call, st...","[19527, 18484, 18167, 18021, 15907, 15173, 144...",0.986097,0.996006


In [25]:
pd.DataFrame(result).to_excel('wiki.xlsx',index=False)